In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

import requests

In [2]:
def raw_data_process(df: pd.DataFrame):
    print("Raw data process in progress.....................")
    '''
    Takes in raw data (in DataFrame),
    returns X (pd.DataFrame), y (pd.Series)
    where X has unused columns removed,
    and the features are the same as the one generated from user input.
    '''
    X = df.drop(columns = ['resale_price', 'block', 'street_name', 'lease_commence_date'])
    y = df['resale_price']

    # X is left with month, town, flat_type,
    # storey_range (string), floor_area_sqm, flat_model, remaining_lease
    # To get to the same features as user input, we have to process storey_range

    X['storey'] = X['storey_range'].apply(lambda x: int(x[:2]))
    X.drop(columns = ['storey_range'], inplace = True)

    X['sale_year'] = X['month'].dt.year
    X['sale_month'] = X['month'].dt.month

    X.drop(columns = ['month'], inplace = True)

    return X, y

In [3]:
def preprocessor(X: pd.DataFrame):
    print("Predicted data process in progress.....................")

    '''
    Takes in DataFrame, returns processed data.
    Transforms a dataset with 7 features into a dataset with 60 features.

    Stateless operation.
    '''
    # Convert remaining_lease from the format XX years YY months to a float (years)
    X['r_lease'] = X['remaining_lease'].apply(lambda x: (int(x[:2])) if x[9:11]== '0 ' or x[9:11] == '' else (int(x[:2]) + int(x[9:11])/12))
    X.drop(columns = ['remaining_lease'], inplace = True)

    #extract year and month details from month column and drop the month column
    X['sale_month_sin'] = np.sin(2 * np.pi * X.sale_month/12)
    X['sale_month_cos'] = np.cos(2 * np.pi * X.sale_month/12)

    X.drop(columns = ['sale_month'], inplace = True)

    def create_preprocessor() -> ColumnTransformer:
        # One hot encoding for categorical features
        town_list = ['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
        'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
        'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
        'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL',
        'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES',
        'TOA PAYOH', 'WOODLANDS', 'YISHUN']

        flat_type_list = ['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '1 ROOM',
            'MULTI-GENERATION']

        flat_model_list = ['Improved', 'New Generation', 'DBSS', 'Standard', 'Apartment',
            'Simplified', 'Model A', 'Premium Apartment', 'Adjoined flat',
            'Model A-Maisonette', 'Maisonette', 'Type S1', 'Type S2',
            'Model A2', 'Terrace', 'Improved-Maisonette', 'Premium Maisonette',
            'Multi Generation', 'Premium Apartment Loft', '2-room', '3Gen']

        categorical_features_names = [np.array(town_list, dtype=object),
                                    np.array(flat_type_list, dtype=object),
                                    np.array(flat_model_list, dtype=object)]

        categorical_ohe = OneHotEncoder(
            categories = categorical_features_names,
            handle_unknown = "ignore",
            sparse = False
            )

        categorical_features = ['town', 'flat_type', 'flat_model']

        # Numerical scalars

        # Floor Area min/max
        f_area_min = 0
        f_area_max = 260

        # Storey min/max
        storey_min = 1
        storey_max = 55

        # Remaining lease years min/max
        r_lease_min = 0
        r_lease_max = 99

        f_area_pipe = FunctionTransformer(lambda p: (p - f_area_min) / (f_area_max - f_area_min))
        storey_pipe = FunctionTransformer(lambda p: (p - storey_min) / (storey_max - storey_min))
        r_lease_pipe = FunctionTransformer(lambda p: (p - r_lease_min) / (r_lease_max - r_lease_min))

        # Combined preprocessor
        final_preprocessor = ColumnTransformer(
                [
                    ("f_area_scalar", f_area_pipe, ["floor_area_sqm"]),
                    ("storey_scalar", storey_pipe, ["storey"]),
                    ("r_lease_scalar", r_lease_pipe, ["r_lease"]),
                    ("categorical", categorical_ohe, categorical_features)
                ],
                n_jobs=-1,
                remainder='passthrough'
            )
        return final_preprocessor

    preprocessor = create_preprocessor()
    X_processed = preprocessor.fit_transform(X)


    return X_processed

In [83]:
ori_df=pd.read_csv('../raw_data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')
flat_coor_df=pd.read_csv('../raw_data/flat_coordinates_clean.csv')
flat_mrt_df=pd.read_csv('../raw_data/flat_mrt.csv')
mrt_coor_df=pd.read_csv('../raw_data/MRT_coordinates.csv')
hdb_coor_df=pd.read_csv('../raw_data/hdb_coor.csv')
flat_school_df=pd.read_csv('../raw_data/flat_school.csv')


In [71]:
ori_df['Address']=ori_df.loc[:,'block']+ " "+ori_df.loc[:,'street_name']
ori_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Address
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,108 ANG MO KIO AVE 4
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,602 ANG MO KIO AVE 5
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,465 ANG MO KIO AVE 10
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,601 ANG MO KIO AVE 5
...,...,...,...,...,...,...,...,...,...,...,...,...
157092,2023-07,YISHUN,5 ROOM,504B,YISHUN ST 51,04 TO 06,113.0,Improved,2016,91 years 09 months,645000.0,504B YISHUN ST 51
157093,2023-07,YISHUN,5 ROOM,613,YISHUN ST 61,07 TO 09,121.0,Improved,1987,62 years 10 months,610000.0,613 YISHUN ST 61
157094,2023-07,YISHUN,5 ROOM,713,YISHUN ST 71,04 TO 06,122.0,Improved,1987,63 years 01 month,535000.0,713 YISHUN ST 71
157095,2023-07,YISHUN,5 ROOM,758,YISHUN ST 72,04 TO 06,129.0,Improved,1986,62 years 05 months,620000.0,758 YISHUN ST 72


In [72]:
hdb_coor_df=hdb_coor_df[['Address','Latitude','Longitude']]
hdb_coor_df

,Address,Latitude,Longitude
0,1 BEACH RD,1.295097,103.854068
1,1 BEDOK STH AVE 1,1.320852,103.933721
2,1 CANTONMENT RD,1.275499,103.841398
3,1 CHAI CHEE RD,1.327969,103.922716
4,1 CHANGI VILLAGE RD,1.388547,103.987805
...,...,...,...
12716,998A BUANGKOK CRES,1.383758,103.880172
12717,998B BUANGKOK CRES,1.384101,103.880505
12718,999 BUANGKOK CRES,1.384564,103.879946
12719,999A BUANGKOK CRES,1.384855,103.880374


In [73]:
ori_df=ori_df.merge(hdb_coor_df, on="Address")

In [74]:
ori_df.tail()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Address,Latitude,Longitude
157092,2023-07,SEMBAWANG,4 ROOM,362A,SEMBAWANG CRES,04 TO 06,93.0,Model A,2019,94 years 11 months,550000.0,362A SEMBAWANG CRES,1.447195,103.814008
157093,2023-06,SEMBAWANG,4 ROOM,366B,SEMBAWANG CRES,10 TO 12,93.0,Model A,2019,94 years 11 months,568000.0,366B SEMBAWANG CRES,1.443922,103.815958
157094,2023-07,SEMBAWANG,4 ROOM,366B,SEMBAWANG CRES,13 TO 15,93.0,Model A,2019,94 years 11 months,565000.0,366B SEMBAWANG CRES,1.443922,103.815958
157095,2023-07,HOUGANG,4 ROOM,992B,BUANGKOK LINK,07 TO 09,93.0,Model A,2019,95 years,598000.0,992B BUANGKOK LINK,1.383827,103.881672
157096,2023-07,SENGKANG,4 ROOM,351A,ANCHORVALE RD,10 TO 12,92.0,Model A,2019,94 years 11 months,568000.0,351A ANCHORVALE RD,1.394153,103.886574


In [75]:

for i,j in mrt_coor_df.iterrows():
    print (j['Latitude'],j['Longitude'],j['STN_NAME'])

1.440585001 103.8009982 ADMIRALTY MRT STATION
1.316432612 103.882893 ALJUNIED MRT STATION
1.369933175 103.8495535 ANG MO KIO MRT STATION
1.388092704 103.9054179 BAKAU LRT STATION
1.380017897 103.7726667 BANGKIT LRT STATION
1.342828338 103.8797462 BARTLEY MRT STATION
1.281873788 103.8590733 BAYFRONT MRT STATION
1.281873788 103.8590733 BAYFRONT MRT STATION
1.341223176 103.77581 BEAUTY WORLD MRT STATION
1.323979969 103.9299587 BEDOK MRT STATION
1.334742117 103.9179554 BEDOK NORTH MRT STATION
1.33660783 103.9322077 BEDOK RESERVOIR MRT STATION
1.29886427 103.8503799 BENCOOLEN MRT STATION
1.313672233 103.8629702 BENDEMEER MRT STATION
1.35130868 103.84915 BISHAN MRT STATION
1.350838988 103.8481398 BISHAN MRT STATION
1.319395706 103.861679 BOON KENG MRT STATION
1.338604054 103.7060994 BOON LAY MRT STATION
1.322423979 103.8161362 BOTANIC GARDENS MRT STATION
1.322110193 103.8149876 BOTANIC GARDENS MRT STATION
1.340471684 103.8467942 BRADDELL MRT STATION
1.296861687 103.8506629 BRAS BASAH MRT STA

In [76]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c  # Earth's radius in kilometers

    return distance

def find_nearest_mrt_station(house_lat, house_lon, mrt_stations):
    nearest_station = None
    min_distance = float('inf')  # Set initial minimum distance to a large value

    for index,station in mrt_stations.iterrows():
        station_lat, station_lon = station['Latitude'], station['Longitude']
        distance = haversine_distance(house_lat, house_lon, station_lat, station_lon)

        if distance < min_distance:
            min_distance = distance
            nearest_station = station['STN_NAME']

    return nearest_station, min_distance

# Example usage:
house_lat = 1.443922	  # Replace with the latitude of your house
house_lon = 103.815958    # Replace with the longitude of your house

mrt_stations = mrt_coor_df

nearest_station, distance = find_nearest_mrt_station(house_lat, house_lon, mrt_stations)
print("Nearest MRT station:", nearest_station)
print("Distance (in kilometers):", distance)


Nearest MRT station: SEMBAWANG MRT STATION
Distance (in kilometers): 0.7295088102749182


In [77]:
ori_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Address,Latitude,Longitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,1.362005,103.853880
1,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 01 month,235000.0,406 ANG MO KIO AVE 10,1.362005,103.853880
2,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,202000.0,406 ANG MO KIO AVE 10,1.362005,103.853880
3,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,210000.0,406 ANG MO KIO AVE 10,1.362005,103.853880
4,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,07 TO 09,44.0,Improved,1979,60 years 01 month,220000.0,406 ANG MO KIO AVE 10,1.362005,103.853880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157092,2023-07,SEMBAWANG,4 ROOM,362A,SEMBAWANG CRES,04 TO 06,93.0,Model A,2019,94 years 11 months,550000.0,362A SEMBAWANG CRES,1.447195,103.814008
157093,2023-06,SEMBAWANG,4 ROOM,366B,SEMBAWANG CRES,10 TO 12,93.0,Model A,2019,94 years 11 months,568000.0,366B SEMBAWANG CRES,1.443922,103.815958
157094,2023-07,SEMBAWANG,4 ROOM,366B,SEMBAWANG CRES,13 TO 15,93.0,Model A,2019,94 years 11 months,565000.0,366B SEMBAWANG CRES,1.443922,103.815958
157095,2023-07,HOUGANG,4 ROOM,992B,BUANGKOK LINK,07 TO 09,93.0,Model A,2019,95 years,598000.0,992B BUANGKOK LINK,1.383827,103.881672


In [80]:
ori_df['nearest_mrt_station']=''
ori_df['distant_nearest_mrt']=''

for i,j in ori_df.iterrows():
    nearest_station, distance = find_nearest_mrt_station(j['Latitude'], j['Longitude'], mrt_stations)
    ori_df.loc[i,'nearest_mrt_station']=nearest_station
    ori_df.loc[i,'distant_nearest_mrt']=distance
    if i in [1000,5000,10000,20000,50000,100000]:
        print(i)
    
    
    
    


1000
5000
10000
20000
50000
100000


In [81]:
ori_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Address,Latitude,Longitude,nearest_mrt_station,distant_nearest_mrt
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,1.362005,103.853880,ANG MO KIO MRT STATION,1.004272
1,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 01 month,235000.0,406 ANG MO KIO AVE 10,1.362005,103.853880,ANG MO KIO MRT STATION,1.004272
2,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,202000.0,406 ANG MO KIO AVE 10,1.362005,103.853880,ANG MO KIO MRT STATION,1.004272
3,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,210000.0,406 ANG MO KIO AVE 10,1.362005,103.853880,ANG MO KIO MRT STATION,1.004272
4,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,07 TO 09,44.0,Improved,1979,60 years 01 month,220000.0,406 ANG MO KIO AVE 10,1.362005,103.853880,ANG MO KIO MRT STATION,1.004272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157092,2023-07,SEMBAWANG,4 ROOM,362A,SEMBAWANG CRES,04 TO 06,93.0,Model A,2019,94 years 11 months,550000.0,362A SEMBAWANG CRES,1.447195,103.814008,SEMBAWANG MRT STATION,0.702661
157093,2023-06,SEMBAWANG,4 ROOM,366B,SEMBAWANG CRES,10 TO 12,93.0,Model A,2019,94 years 11 months,568000.0,366B SEMBAWANG CRES,1.443922,103.815958,SEMBAWANG MRT STATION,0.729568
157094,2023-07,SEMBAWANG,4 ROOM,366B,SEMBAWANG CRES,13 TO 15,93.0,Model A,2019,94 years 11 months,565000.0,366B SEMBAWANG CRES,1.443922,103.815958,SEMBAWANG MRT STATION,0.729568
157095,2023-07,HOUGANG,4 ROOM,992B,BUANGKOK LINK,07 TO 09,93.0,Model A,2019,95 years,598000.0,992B BUANGKOK LINK,1.383827,103.881672,TONGKANG LRT STATION,0.768356


# Sample for api call
url= f'https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1'
response = requests.get('https://developers.onemap.sg/commonapi/search?searchVal=406 ANG MO KIO AVE 10&returnGeom=Y&getAddrDetails=Y&pageNum=1')


In [ ]:
# count=0
# ori_df['lat']=''
# ori_df['lon']=''

# for index,i in enumerate(ori_df['address']):
#     count+=1
#     print(index)
#     url= f'https://developers.onemap.sg/commonapi/search?searchVal={i}&returnGeom=Y&getAddrDetails=Y&pageNum=1'
#     response = requests.get(url)
#     ori_df.loc[index,'lat']=response.json()['results'][0]['LATITUDE']
#     ori_df.loc[index,'lon']=response.json()['results'][0]['LONGITUDE']
#     if count==50:
#         break
    

    
    